In [1]:
#import libraries
import os
import time
import csv
import pandas as pd
import numpy as np
import bs4 as bs
from itertools import product
from urllib.request import Request
import urllib
import sys
import json
import PyPDF2
import collections
import nltk
from collections import Counter
import math
import string
import textmining as txtm


#working directory path
path = 'C:/Users/Lenovo/Desktop/NLP/'
#path = 'C:/Users/user/Python/Natural Language Processsing/'
os.chdir(path)

def download_file(download_url,fname='document.pdf'):
    if sys.version_info[0]==2:
        response = urllib.urlopen(download_url) # python 2
    else:
        response = urllib.request.urlopen(download_url) # python 3

    file = open(fname, 'wb')
    file.write(response.read())
    file.close()
    
#check if the html files contain the word 'privacy' 
#restrict documents that have length less than 100 words
def check_privacy(firm,url,text,min_words=100,domain = ['.com','.gov']):
    if domain[0] not in url:
    #if (domain[0] not in url and domain[1] not in url): 
        flag = "Error:  in url %s corresponding to %s No %s domain" % (url,firm,domain)
    elif ('privacy' not in text.lower()):
        flag = "Error:  in url %s corresponding to %s Does not contain the word privacy" % (url , firm )
    elif len(str(text).split()) < min_words:
        flag = "Error : in url %s corresponding to %s Less than %s words" %  (url , firm ,str(min_words)) 
    else:
        flag = "url %s corresponding to %s is _OK_" % (url,firm)
    return flag

In [2]:
# import stopwords from R's 'tm' library
# import state names of US

rsw = pd.read_csv('C:/Users/Lenovo/Desktop/NLP/rstopwords.csv')['x'].values.tolist() 

states = pd.read_csv('C:/Users/Lenovo/Desktop/NLP/states.csv')['State'].values.tolist()

In [3]:
# load urls
with open("C:/Users/Lenovo/Desktop/NLP/compustat_trial__ggl_urls.json",'r') as fp:
    urls = json.load(fp)
   
firms = list(urls.keys())


In [4]:
len(firms)

144

In [5]:
firms = list(urls.keys())[0:105:1]

In [ ]:
# create master dictionary 
scrape_policies = {}
scrape_errors = {}
replace_strings = ["\\xcb\\x9","\\xc5\\x","\\xef\\xac\\x81","\\xef\\xac\\x82","\b",
                   "\\xe2\\x84\\xa2s","\\\\xe2\\\\x84\\\\xa2s","\\', b\\'",
                   "\\\\xc5\\\\xa0","\\\\xe2\\\\x84\\\\xa2","\\xe2\\x84\\xa2"]


# access the text extracted for each firm using indexes
# change in Load_urls tab from the next line to run for all firms.
for firm in firms:
    scrape_policies[firm] = {}
    scrape_errors[firm] = {}

    for index in [0,1,2,3,4]:
        policies = ""
        length = 0
        try:
            path = 'C:/Users/Lenovo/Desktop/NLP/'
            #path = 'C:/Users/user/Python/Natural Language Processsing/'
            os.chdir(path)
            if urls[firm][index].endswith('.pdf'):
                os.chdir('PDF')
                #check if the file is pdf
                fname="%s_%s.pdf" % (firm,str(index))
                download_file(urls[firm][index], fname)
                
                #extract texts from the pdfs
                pdf_file = open(fname, 'rb')
                read_pdf = PyPDF2.PdfFileReader(pdf_file)
                number_of_pages = read_pdf.getNumPages()
                c = collections.Counter(range(number_of_pages))
                content = ""
                for i in c:
                    page = read_pdf.getPage(i)
                    page_content = page.extractText()
                    content = content + ' ' + str(page_content.encode('utf-8'))
                for string in replace_strings:
                    content = content.replace(string,'')   
                
                
                flag = 'Downloaded %s chars from %s document' % (len(str(content)),fname)
                policies = content
            else:
                sauce = urllib.request.urlopen(Request(urls[firm][index], headers={'User-Agent': 'Mozilla/5.0'})).read()
                soup = bs.BeautifulSoup(sauce,'html5lib')
                for i in list(range(1,len(soup.find_all(['p','ul','table'])))):
                    text = soup.find_all(['p','ul','table'])[i].text
                    policies = policies + ' ' + text
                    length= length+len(text)
                flag = "Downloaded %d chars" % length
            
        except Exception as e:
            flag = "Error: " + str(e)
            policies = flag
        finally:
            print('Firm: %s, URL %s, %s' % (firm,str(index),flag))
        if len(policies) != 0:
            scrape_policies[firm][index] = policies
        else:
            #'a an the' has been used only for indexing purpose - will any way be removed for not meeting 100 words criteria
            scrape_policies[firm][index] = 'a an the'
    

Firm: STANDARD MOTOR PRODUCTS , URL 0, Downloaded 3149 chars
Firm: STANDARD MOTOR PRODUCTS , URL 1, Downloaded 10865 chars
Firm: STANDARD MOTOR PRODUCTS , URL 2, Downloaded 5220 chars
Firm: STANDARD MOTOR PRODUCTS , URL 3, Error: HTTP Error 406: Not Acceptable
Firm: STANDARD MOTOR PRODUCTS , URL 4, Downloaded 1514 chars
Firm: STANDEX INTERNATIONAL , URL 0, Downloaded 17945 chars
Firm: STANDEX INTERNATIONAL , URL 1, Downloaded 1819 chars
Firm: STANDEX INTERNATIONAL , URL 2, Downloaded 35127 chars
Firm: STANDEX INTERNATIONAL , URL 3, Downloaded 13102 chars
Firm: STANDEX INTERNATIONAL , URL 4, Downloaded 49848 chars
Firm: BAYER AG , URL 0, Error: 'NoneType' object has no attribute 'next_element'
Firm: BAYER AG , URL 1, Downloaded 18024 chars
Firm: BAYER AG , URL 2, Downloaded 13248 chars
Firm: BAYER AG , URL 3, Downloaded 21718 chars
Firm: BAYER AG , URL 4, Downloaded 3908 chars
Firm: DEUTSCHE LUFTHANSA AG , URL 0, Error: HTTP Error 405: Method Not Allowed
Firm: DEUTSCHE LUFTHANSA AG , UR

In [ ]:
#remove policies that do not contain 100 words (minimum length)
# Also checking whether the text contains 'privacy' or not using the check_privacy function.
 
okurl = {}
for i in firms:
    okurl[i] = {}

for firm in firms:
    position = []
    for index in range(0,5):
        temp = (check_privacy(firm = firm , url = urls[firm][index] , text = scrape_policies[firm][index]))
        if '_OK_' in temp:
            position.append(index)
            print('%s index %s' % (temp , index) )
        okurl[firm] = position
        

In [ ]:
okurl

In [ ]:
#compare texts extracted from both html files and pdfs
#select the one that has the longest length from the list (pdf & html included)
def testFuncNew(text= 'This is the sample text.',sw=rsw):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in sw])
    return text

for firm in firms:
    length = []
    if len(okurl[firm]) != 0:
        #for i in okurl[firm]:
            #length.append(len(testFuncNew(scrape_policies[firm][i])))

        #maxvalue = max(length)
        okurl[firm] = [okurl[firm][0]]
    if len(okurl[firm]) == 0:
        del okurl[firm]

In [ ]:
okurl

In [ ]:
firms = list(okurl.keys())

In [ ]:
#function to be used to extract headers 
def testFuncNew(text= 'This is sample text.',sw=rsw,key = 'e'):
    if key == 'e':
        text = text.lower()
        text = ' '.join([word for word in text.split() if word not in sw])
    else:
        text = text.lower()
        text = ' '.join([word for word in text.split()])
    return text

In [ ]:
#extract and store header of paragraphs
# Dictionary for storing and accessing headers.

Header_policies = []
Paragraph_policies= []
Headers_Lengths = []

#function to be used to extract headers 
def testFuncNew(text= 'This is sample text.',sw=rsw,key = 'e'):
    if key == 'e':
        text = text.lower()
        text = ' '.join([word for word in text.split() if word not in sw])
    else:
        text = text.lower()
        text = ' '.join([word for word in text.split()])
    return text


#loop over the five urls
for firm in firms:
    if len(okurl[firm]) == 0:
        Header_policies.append(['__NO__ __OK__ __URL__'])
        Paragraph_policies.append(['__NO__ __OK__ __URL__'])
        print('%s __NO__ __OK__ __URL__' %firm)
        Headers_Lengths.append(0)

    for index in okurl[firm]:
        policies_h = []
        policies_p = []
        try:                
            if urls[firm][index].endswith('.pdf'):
                flag = 'Document is PDF'
                policies_h.append('Document is PDF')
                policies_p.append('Document is PDF')
                Headers_Lengths.append(0)
            else:
                sauce = urllib.request.urlopen(Request(urls[firm][index], headers={'User-Agent': 'Mozilla/5.0'})).read()
                #read the html files using bs - use 'lxml' is 'html5lib' doesn't work
                soup = bs.BeautifulSoup(sauce,'html5lib')
                headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5','h6'])
                paras = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5','h6','p','ul','table'])
                headers = [header.get_text() for header in headers]
                paras = [para.get_text() for para in paras]
                
                headers = [x for x in headers if x != '']
                #extract headers - isolate bold texts from the main text
                for header_index,header_word in enumerate(headers):
                    headers[header_index] = testFuncNew(text = header_word,key='ne').upper()
                policies_h = headers
                for para_index,para_sentence in enumerate(paras):
                    paras[para_index] = testFuncNew(text = para_sentence,key='ne').lower()
                policies_p = paras
                flag = "Downloaded %d headers" % len(headers)
                Headers_Lengths.append(len(headers))
            
        except Exception as e:
            flag = "Error: " + str(e)
            policies_h.append(flag)
            policies_p.append(flag)
            Headers_Lengths.append(0)
        finally:
            print('Firm: %s, URL %s, %s' % (firm,str(index),flag))
            Header_policies.append(policies_h)
            Paragraph_policies.append(policies_p)

In [ ]:
# Eleminating blank headers 
for index,header in enumerate(Header_policies):
    temp = [x for x in header if x != '']
    Header_policies[index] = temp
    

In [ ]:
Header_policies

In [ ]:
paragraphs = []
headers_paragraphs = []
Firm_Info = []


for firm,firm_name in list(enumerate(firms)):
    alltext = Paragraph_policies[firm]
    headers = Header_policies[firm]
    temp1 = []
    
    n_headers = len(headers)
    n_alltext = len(alltext)
    
    if n_headers == 1:
        paragraphs.append(' '.join(str(e) for e in alltext))
        temp1.append(' '.join(str(e) for e in alltext))
        Firm_Info.append(firm_name)
        
    else:
        for head in list(range(1,n_headers)):
            check1 = headers[head-1]
            check2 = headers[head]
            pos1 = alltext.index(check1.lower())
            pos2 = alltext.index(check2.lower())
            if (pos2 - pos1) > 1:
                paragraphs.append(' '.join(str(e) for e in alltext[(pos1+1):(pos2)]))
                temp1.append(' '.join(str(e) for e in alltext[(pos1+1):(pos2)]))
                Firm_Info.append(firm_name)
            else:
                paragraphs.append("__No_ __text__ __in__ __the__ __header__")
                temp1.append("__No_ __text__ __in__ __the__ __header__")
                Firm_Info.append(firm_name)
            if head==(n_headers-1):
                paragraphs.append(' '.join(str(e) for e in alltext[(pos2+1):]))
                temp1.append(' '.join(str(e) for e in alltext[(pos2+1):]))
                Firm_Info.append(firm_name)

    for head,para in enumerate(temp1):
        temp = [headers[head],para]
        headers_paragraphs.append(temp)


In [ ]:
# Export headers to csv file - to be saved in the working library
df_headers = pd.DataFrame(Header_policies)
df_headers.insert(loc=0, column='firm', value=firms)
df_headers.to_csv("headers.csv")


In [ ]:
#Extract headers and the corresponding paragraphs
headers_paragraphs_df = pd.DataFrame(headers_paragraphs,columns=("Headers","Paragraphs"))
headers_paragraphs_df['Firm_Info'] = Firm_Info
headers_paragraphs_df

In [ ]:
# Selecting Paragrraphs with only 100 or more characters.
len_list = [len(x) for x in headers_paragraphs_df['Paragraphs']]
out_index = [i for i,x in enumerate(len_list) if x<100]
headers_paragraphs_df.drop(headers_paragraphs_df.index[out_index],inplace=True)
headers_paragraphs_df = headers_paragraphs_df.reset_index(drop=True)
headers_paragraphs_df.to_csv("headers_paragraphs.csv")
headers_paragraphs_df

In [ ]:
#This will generate a csv file for output firms.
res = list(set(headers_paragraphs_df['Firm_Info'].tolist()))
with open('Output_firm.csv', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in res:
        writer.writerow([val])

In [ ]:
# Peparing data for creating text file.
File_Headers = headers_paragraphs_df['Headers'].tolist()
File_Paragraphs = headers_paragraphs_df['Paragraphs'].tolist()
ls = headers_paragraphs_df['Firm_Info'].tolist()
File_Firms = [x for i, x in enumerate(ls) if ls.index(x) == i]
temp1 = []
for firm in File_Firms:
    temp = []
    for item in range(0,len(File_Headers)):
            if firm in ls[item]:
                temp.append(item)
    temp1.append(temp)

In [ ]:
#Text file to be saved in the working directory
thefile = open('test.txt', 'w')

for index,firm in enumerate(File_Firms):
    thefile.write("\n# %s \n" %firm)
    #thefile.write("\n--@@@@%s@@@@--\n" %urls[firm][0])
    for i in temp1[index]:
        try:
            thefile.write("\n## %s \n %s \n" % (File_Headers[i],File_Paragraphs[i]))
            print("Header: %s Success"%(File_Headers[i]))
        except Exception as e:
           
                print(e)
        
thefile.close()

In [ ]:
len(okurl)

In [ ]:
#this function(tdm_df) would be used to extract words from the policies - eventually to create TDM matrix
from pandas import *
def tdm_df(doclist, stopwords = [], remove_punctuation = True, 
           remove_digits = True, sparse_df = False):
    
    # Create the TDM from the list of documents.
    tdm = txtm.TermDocumentMatrix()
  
    for doc in doclist:
        if remove_punctuation == True:
            doc = doc.translate(None, string.punctuation.translate(None, '"'))
        if remove_digits == True:
            doc = doc.translate(None, string.digits)
            
        tdm.add_doc(doc)
    
    # Push the TDM data to a list of lists,
    # then make that an ndarray, which then
    # becomes a DataFrame.
    tdm_rows = []
    for row in tdm.rows(cutoff = 1):
        tdm_rows.append(row)
        
    tdm_array = np.array(tdm_rows[1:])
    tdm_terms = tdm_rows[0]
    df = DataFrame(tdm_array, columns = tdm_terms)
    
    # Remove stopwords from the dataset, manually.
    # TermDocumentMatrix does not do this for us.
    if len(stopwords) > 0:
        for col in df:
            if col in stopwords:
                del df[col]
    
    if sparse_df == True:
        df.to_sparse(fill_value = 0)
    
    return df

In [ ]:
#create a dictionary to store data frames for each url 

TDM = {}
for i in firms:
    TDM[i] = {}
    
TDM.keys()



In [ ]:
#extract unigrams for each firm 
#policy that is longest and meets the mimimum criteria is used for each firm

from IPython.display import display
pd.options.display.max_rows = None
 
for firm in firms:
    for index in okurl[firm]:
        TDM[firm][index] = {}
        TDM[firm][index] = tdm_df([scrape_policies[firm][index]],stopwords = rsw,remove_punctuation = False,remove_digits = False)


In [ ]:
# Create a list to store unique firms
master_data=[]
selected_firms=[]

for firm in firms:
    index = okurl[firm]
    if len(index) != 0:
        master_data.append(scrape_policies[firm][okurl[firm][0]])
    #else:
      #  master_data.append('__aemptya__')
    
len(master_data)


In [ ]:
#create term document matrix (TDM) using the master data

master_tdm = tdm_df(master_data,stopwords = rsw,remove_punctuation = False,remove_digits = False)
master_tdm.shape

In [ ]:
#geographic indicator indicates if name(s) of any of the US state(s)
#appear in the document term matrix (TDM) for a given firm 
geography_information = []
geography_indicator = []

#names of the states have been appended - can be seen in the right most column of the output file
for index,firm_data in enumerate(master_data):
    temp = ""
    for state in states:
        if state in firm_data:
            temp = temp + ' ' + state
    if len(temp) != 0:
        geography_information.append(temp)
        geography_indicator.append(1)
    else:
        geography_information.append('__NO__')
        geography_indicator.append(0)

In [ ]:
#assign geographic information to the TDM
master_tdm["geography_indicator"] = geography_indicator
master_tdm['geography_information'] = geography_information

In [ ]:
#export unigram matrix to csv
master_tdm.to_csv("unigram.csv",sep=",")

In [ ]:
#this will prepare data to create bigrams 
#the structure of the bigram file (BDM) will remain same as that of the TDM
def testFuncNew(text= "Hello, I am fine",rsw=rsw):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in rsw])
    return text
#store bigrams
list_data = []

#loop over each firm
for firm in firms:
    length = okurl[firm]
    breaks = firm.lower().split()
    for i in breaks:
        rsw.append(i)
    if len(length) != 0:
        tdm = txtm.TermDocumentMatrix()
        doc = scrape_policies[firm][okurl[firm][0]]
        tdm.add_doc(doc)

        tdm_rows = []
    
        for row in tdm.rows(cutoff = 1):
            tdm_rows.append(row)
        
        tdm_array = np.array(tdm_rows[1:])
        tdm_terms = tdm_rows[0]

        text = ""
        for index , word in enumerate(tdm_terms):
            text = text + " " + word 
        text = testFuncNew(text,rsw = rsw)    
        list_data.append(text)
    else:
        list_data.append("")
list_data

In [ ]:
#use nltk to create consin dissimilarity matrix
#Function for eleminating stopwords and 

counter_sum = Counter()
for line in list_data:
    bigrams = list(nltk.bigrams(line.split()))
    bigramsC = Counter(bigrams)
    counter_sum += bigramsC
#export the bigram matrix to a csv file - to be saved in the working directory

with open('bigram.csv', 'w', newline='') as csvfile:
    header = sorted(counter_sum, key=lambda x: str(type(x)))
    writer = csv.DictWriter(csvfile, fieldnames=header)
    writer.writeheader()
    for line in list_data:
        bigrams = list(nltk.bigrams(line.split()))
        bigramsC = Counter(bigrams)
        cs = dict(counter_sum)
        bc = dict(bigramsC)
        row = {}
        for element in list(cs):
            if element in list(bc):
                row[element] = bc[element]
            else:
                row[element] = 0
        writer.writerow(row)

In [ ]:
#compute cosine similarities 
#values are stored in a matrix - looks similar to a var-covar or correlation matrix

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df=1)
text = vect.fit_transform(list_data)

temp_array = (text * text.T).A


In [ ]:
#export cosine similarities matrix to a csv file
import pandas as pd
# 
df_cosine = pd.DataFrame(temp_array,columns=firms,index=firms)
df_cosine
df_cosine.to_csv("F:/Cosine.csv")

In [ ]:
# Make a worldcloud for the unigrams.
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Generate word cloud image
wordcloud = WordCloud().generate(text)


text = str(master_tdm)

wordcloud = WordCloud(max_font_size=40).generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
# save results to json # if does not work run the very first tab again(import the libraries again) and then run 
path = 'C:/Users/Lenovo/Desktop/NLP/'
with open(path + 'hmda_scraped_policies.json', 'w') as fp:
    json.dump(scrape_policies, fp)